In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import cv2
from deepface import DeepFace

2024-12-18 15:29:28.055289: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-18 15:29:28.134791: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-18 15:29:28.134848: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-18 15:29:28.134933: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-18 15:29:28.151468: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-18 15:29:28.152085: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [8]:
# Ensure results directory exists
os.makedirs("results/plots", exist_ok=True)

# Placeholder paths for test datasets
fer2013_test_data = "FER-2013/test"

In [4]:
# ## Section 2: Load and Preprocess Data
# Define functions for loading and preprocessing FER2013 dataset

def load_fer2013_images(data_dir):
    """
    Load images from FER2013 directories structured by class names.
    :param data_dir: Path to directory containing class folders
    :return: List of tuples (image path, label)
    """
    data = []
    classes = os.listdir(data_dir)
    for label in classes:
        class_dir = os.path.join(data_dir, label)
        if os.path.isdir(class_dir):
            for file in os.listdir(class_dir):
                if file.endswith(('.png', '.jpg', '.jpeg')):
                    data.append((os.path.join(class_dir, file), label))
    print(f"Loaded {len(data)} images from {len(classes)} classes.")
    return data

In [10]:
def validate_deepface_fer2013(image_data, sample_size=100):
    """
    Validate DeepFace on images in FER2013 dataset.
    :param image_data: List of tuples (image path, label)
    :param sample_size: Number of samples to process
    :return: Ground truth and predicted labels
    """
    ground_truth = []
    predictions = []
    
    # Sampling data
    sampled_data = image_data[:sample_size]
    print(f"Validating DeepFace on {len(sampled_data)} samples...")
    
    for img_path, label in tqdm(sampled_data):
        ground_truth.append(label)
        
        try:
            # Load image using OpenCV
            img = cv2.imread(img_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            
            # Analyze image with DeepFace
            result = DeepFace.analyze(img_path=img, actions=['emotion'], enforce_detection=False, silent=True)
            predicted_label = result[0]['dominant_emotion']
        except Exception as e:
            print(f"Error processing {img_path}: {e}")
            predicted_label = "error"
        predictions.append(predicted_label)
    
    return ground_truth, predictions

In [16]:
# ## Section 4: Visualize Results
# Define visualization functions for DeepFace validation results

def plot_confusion_matrix(cm, labels, title, save_path):
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels)
    plt.title(title)
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    
    # Удаляем старый файл, если он существует
    if os.path.exists(save_path):
        os.remove(save_path)

    plt.savefig(save_path)
    plt.close()

def visualize_results(ground_truth, predictions, save_dir):
    """
    Generate confusion matrix and classification report
    :param ground_truth: List of ground truth labels
    :param predictions: List of predicted labels
    :param save_dir: Directory to save plots
    """
    labels = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
    cm = confusion_matrix(ground_truth, predictions, labels=labels)
    print("\nClassification Report:")
    print(classification_report(ground_truth, predictions, labels=labels))
    
    # Save confusion matrix plot
    cm_path = os.path.join(save_dir, "fer2013_confusion_matrix.png")
    plot_confusion_matrix(cm, labels, "DeepFace Validation on FER2013", cm_path)
    print(f"Confusion matrix saved at: {cm_path}")

In [19]:
# ## Section 5: Run Validation on FER2013
print("\nStarting FER2013 validation with DeepFace...")
fer2013_data = load_fer2013_images(fer2013_test_data)
ground_truth, predictions = validate_deepface_fer2013(fer2013_data, sample_size=7000)
visualize_results(ground_truth, predictions, save_dir="results/plots")

print("Validation complete. Results saved.")



Starting FER2013 validation with DeepFace...
Loaded 7178 images from 7 classes.
Validating DeepFace on 7000 samples...


  0%|          | 0/7000 [00:00<?, ?it/s]

100%|██████████| 7000/7000 [03:33<00:00, 32.86it/s]



Classification Report:
              precision    recall  f1-score   support

       angry       0.52      0.43      0.47       958
     disgust       0.62      0.45      0.52       111
        fear       0.40      0.42      0.41      1024
       happy       0.78      0.76      0.77      1774
     neutral       0.47      0.54      0.50      1233
         sad       0.39      0.41      0.40      1069
    surprise       0.77      0.70      0.74       831

    accuracy                           0.56      7000
   macro avg       0.56      0.53      0.54      7000
weighted avg       0.57      0.56      0.56      7000

Confusion matrix saved at: results/plots/fer2013_confusion_matrix.png
Validation complete. Results saved.


In [7]:
# ## Section 6: Summary and Next Steps
# Provide a summary of the validation and propose weight adjustments

print("Notebook initialized. Proceed with loading data and validating models.")

Notebook initialized. Proceed with loading data and validating models.
